In [ ]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [ ]:
backend = provider.get_backend('ibmq_belem')

n_qubits = len(backend.properties().qubits)
n_meas_shots = 16000
n_build_shots = 16000
n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))


n_circuit_method_shots = n_meas_shots + n_build_shots

circuit = GHZ_prep(backend)

err_cmap = [[1, 4], [3, 4], [1, 2], [2, 4], [0, 2]]

## Qiskit Full Build

In [ ]:
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 

full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
n_shots_qiskit_partial = n_build_shots // (n_qubits)

linear_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_partial)


## Run with repetitions
If everything above is working, let's crank it out a few times

In [ ]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[]
}

for _ in range(n_reps):

    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
    tpf_err.build(verbose=True)
    
    tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
    tpf.build(verbose=True)
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        GHZ_state_dist(bare_result)
    )
    results['full'].append(
        GHZ_state_dist(full_filter.apply(bare_result))
    )
    results['linear'].append(
            GHZ_state_dist(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
            GHZ_state_dist(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
            GHZ_state_dist(tpf_err.apply(bare_result))
    )
    results['aim'].append(GHZ_state_dist(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(GHZ_state_dist(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(GHZ_state_dist(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))


In [ ]:
ghz_belem_results = {'bare': [0.19487499999999996,
  0.21581249999999996,
  0.19524999999999998,
  0.20656250000000004,
  0.21712499999999996,
  0.2311875,
  0.20293750000000005,
  0.20756249999999998,
  0.20799999999999996,
  0.20275000000000004],
 'full': [0.05064799702063694,
  0.07301821742337833,
  0.049868722474552485,
  0.06394459721438278,
  0.07552646434697713,
  0.08831166515947392,
  0.06089679254604835,
  0.06626683473705969,
  0.0698420688214621,
  0.0608034187180565],
 'linear': [0.056897143346287926,
  0.0582941403877027,
  0.04558802981835641,
  0.050138068865863916,
  0.062156304339220037,
  0.07225630106002451,
  0.057116188791234646,
  0.05993818160454634,
  0.07867612691273812,
  0.06211356098903936],
 'aim': [0.2245416666666667,
  0.21220833333333333,
  0.22250000000000003,
  0.20470833333333333,
  0.2064166666666667,
  0.2169583333333333,
  0.2064583333333333,
  0.22049999999999997,
  0.21737499999999998,
  0.2224166666666667],
 'sim': [0.23199999999999998,
  0.22825,
  0.2155625,
  0.22084375,
  0.21959374999999998,
  0.2235625,
  0.22165625,
  0.2265625,
  0.22703125000000002,
  0.22671875000000002],
 'jigsaw': [0.13906157469336117,
  0.14219600847796415,
  0.1818358679820128,
  0.14612996164451864,
  0.15543004121252357,
  0.17745196850576844,
  0.17039931848017303,
  0.14369600050287468,
  0.1728469868622644,
  0.15180958727886196],
 'cmc': [0.17445453934232796,
  0.17112034822364508,
  0.16578350604889291,
  0.18005355163536252,
  0.19030640999014709,
  0.189922419521804,
  0.1794927324300063,
  0.18282781853161278,
  0.1915177818948024,
  0.17787583942127677],
 'cmc_err': [0.2169099185768521,
  0.16309790731937202,
  0.2175351864561625,
  0.20351470194093452,
  0.19580875174155765,
  0.1823843448796933,
  0.20742364187777185,
  0.19908652762094547,
  0.22110692154486394,
  0.17856245205411486]}

In [ ]:
import numpy as np
res = ghz_belem_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')
